I have run a number of experiments: 

```bash
python score.py --maxevals 50 --save
python score.py --maxevals 50 --with_bigrams --save
python score.py --packg spacy --maxevals 50 --save
python score.py --packg spacy --maxevals 50 --with_bigrams --save

python score.py --algo lda --n_topics 20 --with_cv --save
python score.py --algo lda --n_topics 50 --with_cv --save
python score.py --algo lda --n_topics 20 --with_bigrams --with_cv --save
python score.py --algo lda --n_topics 50 --with_bigrams --with_cv --save
python score.py --packg spacy --algo lda --n_topics 20 --with_cv --save
python score.py --packg spacy --algo lda --n_topics 50 --with_cv --save
python score.py --packg spacy --algo lda --n_topics 20 --with_bigrams --with_cv --save
python score.py --packg spacy --algo lda --n_topics 50 --with_bigrams --with_cv --save

python score.py --algo ensemb --n_topics 20 --with_cv --save
python score.py --algo ensemb --n_topics 20 --with_bigrams --with_cv --save
python score.py --packg spacy --algo ensemb --n_topics 20 --with_cv --save
python score.py --packg spacy --algo ensemb --n_topics 20 --with_bigrams --with_cv --save
```

I have used LDA, EnStop (pLSA+UMAP) and tfidf with nltk and spacy tokenization (see `preprocessing.py`) with and without bigrams. 

All the code neccesary to run the experiments can be found in `score.py`. Is mostly this:

In [7]:
import numpy as np
import pickle
import argparse
import pdb

from pathlib import Path
# Note: I simply copied the `utils` dir into the notebooks dir so that I can run the next cell here
from utils.lightgbm_optimizer import LGBOptimizer

In [22]:
packg = 'nltk'           # nltk or spacy
with_bigrams = False
algo = 'lda'             # lda or ensemb
n_topics = '20'          # 20 or 50 when lda, only 20 for ensemb
with_cv = False          # hyperoptimize with cross validation
is_unbalance = True      # set the lightgbm is_unbalance param to True/False
with_focal_loss = False  # Use the Focal Loss (see here: https://github.com/jrzaurin/LightGBM-with-Focal-Loss)
eval_with_metric = False # hyperoptimize using the F1 score or the CE Loss
save = False             
maxevals = 1

In [23]:
FEAT_PATH = Path('../features')

wbigram = 'bigram_' if with_bigrams else ''
dataname = packg + '_tok_reviews_' + wbigram + algo
if algo is not 'tfidf': dataname = dataname + '_' +  n_topics

dtrain = pickle.load(open(FEAT_PATH/('train/'+ dataname+'_feat_tr.p'), 'rb'))
dvalid = pickle.load(open(FEAT_PATH/('valid/'+ dataname+'_feat_val.p'), 'rb'))
dtest  = pickle.load(open(FEAT_PATH/('test/' + dataname+'_feat_te.p'),  'rb'))

opt = LGBOptimizer(
    dataname,
    dtrain,
    dvalid,
    dtest,
    with_cv=with_cv,
    is_unbalance=is_unbalance,
    with_focal_loss=with_focal_loss,
    eval_with_metric=eval_with_metric,
    save=save)
opt.optimize(maxevals=maxevals)

100%|██████████| 1/1 [00:04<00:00,  4.02s/it, best loss: 0.9605855850150624]
acc: 0.6088, f1 score: 0.5341, precision: 0.5264, recall: 0.6088
confusion_matrix
[[  652   172   380  1461]
 [  398   196   532  1916]
 [  361   185   762  4527]
 [  284   130   555 15353]]


Let's first comment a bit on what happens within `LGBOptimizer`. Here, I use `hyperopt` and the following parameter space: 

```python
space = {
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.2),
    'num_boost_round': hp.quniform('num_boost_round', 50, 500, 20),
    'num_leaves': hp.quniform('num_leaves', 31, 255, 4),
    'min_child_weight': hp.uniform('min_child_weight', 0.1, 10),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.),
    'subsample': hp.uniform('subsample', 0.5, 1.),
    'reg_alpha': hp.uniform('reg_alpha', 0.01, 0.1),
    'reg_lambda': hp.uniform('reg_lambda', 0.01, 0.1),
}
```

To optimize LightGBM hyper-parameters. I think I could perhaps refine a bit more some of them, like using a slightly higher `learning_rate` (e.g 0.3) of a smaller `num_boost_round` (e.g. 10). I will leave that to future visits to this repo or to you, the reader 🙂. 

Within the class `LGBOptimizer`, one can run the optimization process with a number of options. For example, using cross validation and the F1 score. This will basically run the next piece of code:

```python
cv_result = lgb.cv(
    params,
    dtrain,
    num_boost_round=params['num_boost_round'],
    metrics='multi_logloss',
    feval = lgb_f1 if self.eval_with_metric else None,
    nfold=3,
    stratified=True,
    early_stopping_rounds=20)
```

where lgb_f1 is

```python
def lgb_f1_score(preds, lgbDataset, num_class):
	"""
	Implementation of the f1 score to be used as evaluation score for lightgbm
	Parameters:
	-----------
	preds: numpy.ndarray
		array with the predictions
	lgbDataset: lightgbm.Dataset
	"""
	preds = preds.reshape(-1, num_class, order='F')
	cat_preds = np.argmax(preds, axis=1)
	y_true = lgbDataset.get_label()
	return 'f1', f1_score(y_true, cat_preds, average='weighted'), True

```

If you want to know more about custom losses and metrics for lightgbm, have a look to my repo [here](https://github.com/jrzaurin/LightGBM-with-Focal-Loss). All this can be found in the `utils` module. 

So, if we choose to run 100 iterations, with cross validation and evaluate using the F1 score, for a set of features that is the results of using LDA with 20 topics, `LGBOptimizer` will take the train, validation and test datasets, merge the first two and run a 3 stratified-fold CV experiments on them. Once the best parameters have been found based on the resulting F1 score, it will then predict on the test set and compute the success metrics. 

```python
acc  = accuracy_score(self.lgtest.label, preds)
f1   = f1_score(self.lgtest.label, preds, average='weighted')
prec = precision_score(self.lgtest.label, preds, average='weighted')
rec  = recall_score(self.lgtest.label, preds, average='weighted')
cm   = confusion_matrix(self.lgtest.label, preds)
```

and "that's it". Let's have a look to the results for the 16 experiments at the top of this notebook:

In [42]:
RESULT_PATH = Path('../results/')

In [43]:
results_fnames = list(RESULT_PATH.glob("*.p"))

In [44]:
rnames = [str(rf).replace('../results/', '') for rf in results_fnames]
rnames = [str(rf).replace('_results_unb.p', '') for rf in rnames]